## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Maragogi,BR,-9.0122,-35.2225,85.93,52,28,14.36,scattered clouds
1,1,Yellowknife,CA,62.4560,-114.3525,-10.44,76,75,16.11,broken clouds
2,2,Barrow,US,71.2906,-156.7887,-27.38,68,0,5.75,clear sky
3,3,Port Alfred,ZA,-33.5906,26.8910,72.27,78,17,12.26,few clouds
4,4,Rikitea,PF,-23.1203,-134.9692,78.64,74,18,18.01,few clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip?"))
max_temp = float(input("What is the maximum temperature you would like for your trip?"))

What is the minimum temperature you would like for your trip?70
What is the maximum temperature you would like for your trip?90


In [23]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df['Max Temp'] <= max_temp) & \
                                       (city_data_df['Max Temp'] >= min_temp)]

In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

City_ID                0
City                   0
Country                4
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [21]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = city_data_df.loc[(city_data_df['Max Temp'] <= max_temp) & (city_data_df['Max Temp'] >= min_temp)].dropna()
clean_df.count()

City_ID                270
City                   270
Country                270
Lat                    270
Lng                    270
Max Temp               270
Humidity               270
Cloudiness             270
Wind Speed             270
Current Description    270
dtype: int64

In [25]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Maragogi,BR,85.93,scattered clouds,-9.0122,-35.2225,
3,Port Alfred,ZA,72.27,few clouds,-33.5906,26.8910,
4,Rikitea,PF,78.64,few clouds,-23.1203,-134.9692,
5,Manta,EC,78.58,broken clouds,-0.9500,-80.7333,
6,Butaritari,KI,81.27,light rain,3.0707,172.7902,
22,Talcahuano,CL,71.28,clear sky,-36.7167,-73.1167,
23,Pemba,MZ,79.50,broken clouds,-12.9740,40.5178,
29,Puerto Ayora,EC,82.36,broken clouds,-0.7393,-90.3518,
31,Lorengau,PG,80.38,broken clouds,-2.0226,147.2712,
32,Kudahuvadhoo,MV,81.46,broken clouds,2.6708,72.8944,


In [28]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # 6e. Make request and retrieve the JSON data from the search. 
   
    hotels = requests.get(base_url, params = params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    
    except (IndexError):
        print ("Hotel not found... skipping")

print("Done.")
        

Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Done.


In [50]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna(subset = ["Hotel Name"])
clean_hotel_df["Hotel Name"].count()

270

In [49]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# Export the City_Data into a CSV.
city_data_df.to_csv(output_data_file, index_label="City_ID")
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [ ]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """

"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [ ]:
# 11a. Add a marker layer for each city to the map. 

# 11b. Display the figure
